In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-23"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
20469,2024-03-23,Polônia Liga De Basquete,15:30,Dziki Warszawa,Anwil Wloclawek,2.51,1.50,158.5,1.87,1.79,0.0,0.0,0.0,MP9vEhz9,0.398406,0.666667,0.534759,0.558659,0.065073,168.988,58.436341,0.345802,1.8,1.643168,0.912871,271.44,2.222,0.876567,0.394495,-4.0,102.898,14.029559,0.136344,1.2,1.643168,1.369306,81.75,1.204,0.144845,0.120303,-1.0,72,75,3.77,1.09,167.600,118.602,0.356198,0.030912,NaN,1.11,0.222,6.801802,0.527282,0.5,-0.027282,-0.86,-0.172,-2.906977,0.000000,0.0,0.000000
20470,2024-03-23,Polônia Liga De Basquete,13:30,Trefl Sopot,Gdynia,1.10,6.48,171.5,1.81,1.85,0.0,0.0,0.0,jaErDCkF,0.909091,0.154321,0.552486,0.540541,0.063412,132.626,35.121368,0.264815,2.4,1.341641,0.559017,134.30,1.558,0.388356,0.249265,44.0,350.196,125.623987,0.358725,1.2,1.643168,1.369306,231.30,4.100,1.785441,0.435473,-27.0,85,90,1.58,2.57,125.736,310.512,1.003756,0.015456,NaN,0.97,0.194,0.515464,0.751751,0.7,-0.051751,-5.00,-1.000,-5.480000,0.000000,0.0,0.000000
20471,2024-03-23,Polônia Liga De Basquete,10:30,Spojnia Stargard,Slask Wroclaw,1.57,2.34,157.5,1.88,1.79,0.0,0.0,0.0,rZAzFYK2,0.636943,0.427350,0.531915,0.558659,0.064293,151.394,62.311090,0.411582,1.8,1.643168,0.912871,99.76,1.864,0.874174,0.468977,28.0,147.376,60.230131,0.408683,2.4,1.341641,0.559017,95.14,1.802,0.684047,0.379604,12.0,86,67,1.16,1.42,111.626,163.768,0.278502,0.034681,NaN,0.24,0.048,11.875000,0.705996,0.7,-0.005996,1.35,0.270,4.962963,0.516947,0.6,0.083053
20472,2024-03-23,Noruega Blno,12:00,Bærum,Kongsberg Miners,2.68,1.43,163.5,1.80,1.86,0.0,0.0,0.0,WpGYVl9l,0.373134,0.699301,0.555556,0.537634,0.072435,205.726,195.160106,0.948641,3.0,0.000000,0.000000,553.92,2.298,1.961140,0.853412,50.0,116.610,22.356406,0.191719,2.4,1.341641,0.559017,99.90,1.150,0.128258,0.111528,101.0,96,90,5.77,1.11,181.356,116.522,0.430114,0.023184,NaN,-1.13,-0.226,-7.433628,0.706203,0.8,0.093797,0.97,0.194,2.216495,0.862387,0.9,0.037613
20473,2024-03-23,México Cibacopa,00:30,Caballeros de Culiacan,Frayles de Guasave,1.56,2.32,176.5,1.85,1.85,0.0,0.0,0.0,lApyNvCJ,0.641026,0.431034,0.540541,0.540541,0.072060,232.106,109.887362,0.473436,0.0,0.000000,NaN,285.93,2.758,1.158758,0.420144,-48.0,189.676,35.440927,0.186850,0.0,0.000000,NaN,218.05,2.158,0.387130,0.179393,-62.0,81,89,3.53,2.45,121.260,0.000,0.277011,0.000000,NaN,-3.74,-0.748,-0.748663,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20739,2024-03-23,Eua Ncaa,16:15,Kansas,Gonzaga,2.56,1.55,151.5,1.91,1.91,3.5,2.0,1.3,Gd4G3sEj,0.390625,0.645161,0.523560,0.523560,0.035786,145.028,39.851531,0.274785,1.2,1.643168,1.369306,124.62,2.276,1.124558,0.494094,-32.0,123.482,24.555480,0.198859,2.4,1.341641,0.559017,114.38,1.612,0.415114,0.257515,55.0,93,86,1.34,1.33,111.012,116.632,0.347532,0.000000,0.299985,-1.14,-0.228,-6.842105,0.719913,0.7,-0.019913,0.99,0.198,2.777778,0.757648,0.8,0.042352
20740,2024-03-23,Eua Ncaa,13:45,Arizona,Dayton,1.21,4.67,148.5,1.80,1.95,-10.5,2.0,11.0,vZPbppfn,0.826446,0.214133,0.555556,0.512821,0.040579,87.554,14.464546,0.165207,1.8,1.643168,0.912871,87.55,1.198,0

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
20508,09:30,Indonésia Ibl,Tangerang Hawks,Hangtuah,1.55,Back Home
20526,03:35,Japão B.League,Nagoya Fighting Eagles,Shimane,2.26,Back Home
20539,11:30,Turquia Kbsl Feminina,Izmit Bld. F,Galatasaray F,2.53,Back Home
20601,06:00,Taiwan Sbl,Bank of Taiwan,Yulon Luxgen Dinos,1.76,Back Home
20617,15:30,Bósnia E Herzegovina Prvenstvo Bih,Slavija,Orlovik,1.66,Back Home
20658,20:00,Argentina Liga A,Platense,Boca Juniors,2.07,Back Home
20671,16:30,Itália Liga A,Trento,Sassari,1.71,Back Home
20686,12:00,Finlândia Korisliiga,Kobrat,KTP Basket,1.57,Back Home
20689,12:00,Finlândia Korisliiga Feminina,Pyrinto F,Forssan Alku F,2.19,Back Home
20727,16:15,Espanha Leb Plata,Huesca,Ciudad de Huelva,1.56,Back Home
